# Troy&AbedInTheMorning Team Participation in SMM4H-Spanish

Team Participants:
* Sergio Santamaria Carrasco
* Roberto Cuervo Rosillo

## Settings

1. Setting the parent dirrectory
2. Checking GPU is avaible

In [1]:
sst_home = '/home/sergio/Escritorio/ProfNER'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12994388458322486098
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15057625033811068942
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3547922432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7148418811094183583
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4373140510700367521
physical_device_desc: "device: XLA_GPU device"
]


## Requirements

    Tensorflow 1.14.0
    Keras 2.2.4
    Fasttext 0.2.0
    MeaningCloud-python
    Spacy
    Sklearn_crfsuite
    Keras-contrib
    Pandas

## Text Format

The data of the corpus was obtained from a Twitter crawl that used keywords like “Covid-19”, “epidemia” (epidemic) or “confinamiento” (lockdown), as well as hashtags such as “#yomequedoencasa” (#istayathome), to retrieve relevant tweets.

In [4]:
file_text = sst_home + '/profner-data/subtask-2/brat/train/1242399976644325376.txt' 

with open(file_text, 'r') as file:
  print(file.read())

Cerramos nuestra querida Radio 😢 Nuestros colaboradores y conductores ¡Se quedan en casa! Desde mañana todos los programas de Radio Hoy se harán vía remoto X Skype. Seguimos al aire con el compromiso de siempre, nosotros apoyamos el #QuedateEnCasa #covıd19 #Coronavirus #RadioHoy https://t.co/Q81BBYpbdM


## Annotations Format

The corpus was annotated by linguist experts in an iterative process that included the creation of annotation guidelines specifically for this task. We could find the data annotated by BRAT and BIO format.

In [5]:
file_ann = sst_home + '/profner-data/subtask-2/brat/train/1242399976644325376.ann' 

with open(file_ann) as file:
  print(file.read())

T1	PROFESION 42 55	colaboradores
T2	PROFESION 58 69	conductores



## Requirements

TODO: Incluir los diferentes requerimientos del sistema

## Syllabizer

Using the rules of Spanish orthography, the syllabizer allows a word to be broken into its component syllables. The necessary code has been extracted from https://github.com/mabodo/sibilizador/blob/master/Silabizator.ipynb

In [6]:
class char():
    def __init__(self):
        pass
    
class char_line():
    def __init__(self, word):
        self.word = word
        self.char_line = [(char, self.char_type(char)) for char in word]
        self.type_line = ''.join(chartype for char, chartype in self.char_line)
        
    def char_type(self, char):
        if char in set(['a', 'á', 'e', 'é','o', 'ó', 'í', 'ú']):
            return 'V' #strong vowel
        if char in set(['i', 'u', 'ü']):
            return 'v' #week vowel
        if char=='x':
            return 'x'
        if char=='s':
            return 's'
        else:
            return 'c'
            
    def find(self, finder):
        return self.type_line.find(finder)
        
    def split(self, pos, where):
        return char_line(self.word[0:pos+where]), char_line(self.word[pos+where:])
    
    def split_by(self, finder, where):
        split_point = self.find(finder)
        if split_point!=-1:
            chl1, chl2 = self.split(split_point, where)
            return chl1, chl2
        return self, False
     
    def __str__(self):
        return self.word
    
    def __repr__(self):
        return repr(self.word)

class silabizer():
    def __init__(self):
        self.grammar = []
        
    def split(self, chars):
        rules  = [('VV',1), ('cccc',2), ('xcc',1), ('ccx',2), ('csc',2), ('xc',1), ('cc',1), ('vcc',2), ('Vcc',2), ('sc',1), ('cs',1),('Vc',1), ('vc',1), ('Vs',1), ('vs',1)]
        for split_rule, where in rules:
            first, second = chars.split_by(split_rule,where)
            if second:
                if first.type_line in set(['c','s','x','cs']) or second.type_line in set(['c','s','x','cs']):
                    #print 'skip1', first.word, second.word, split_rule, chars.type_line
                    continue
                if first.type_line[-1]=='c' and second.word[0] in set(['l','r']):
                    continue
                if first.word[-1]=='l' and second.word[-1]=='l':
                    continue
                if first.word[-1]=='r' and second.word[-1]=='r':
                    continue
                if first.word[-1]=='c' and second.word[-1]=='h':
                    continue
                return self.split(first)+self.split(second)
        return [chars]
        
    def __call__(self, word):
        return self.split(char_line(word))

## Spacy Model

The Spacy model selected is 'es_core_news_md'. Cause this model doesn't tokenize hashtags, is necessarily keep the attention during preprocessing step.

In [7]:
!python -m spacy download es_core_news_md

import spacy
import re
import es_core_news_md

nlp = es_core_news_md.load()


✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [8]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Token

# We're using a class because the component needs to be initialised with
# the shared vocab via the nlp object
class HashtagMerger(object):
    def __init__(self, nlp):
        # Register a new token extension to flag bad HTML
        Token.set_extension("is_hashtag", default=False)
        self.matcher = Matcher(nlp.vocab)
        # Add pattern for valid hashtag, i.e. '#' plus any ASCII token
        self.matcher.add("HASHTAG", None, [{"ORTH": "#"}, {"TEXT": {"REGEX": "[A-Za-záéíóúÁÉÍÓÚ]+"}}])

    def __call__(self, doc):
        # This method is invoked when the component is called on a Doc
        matches = self.matcher(doc)
        spans = []  # Collect the matched spans here
        for match_id, start, end in matches:
            spans.append(doc[start:end])
        with doc.retokenize() as retokenizer:
            for span in spans:
                retokenizer.merge(span)
                for token in span:
                    token._.is_hashtag = True  # Mark token as bad HTML
        return doc
    
hashtag_merger = HashtagMerger(nlp)
nlp.add_pipe(hashtag_merger, last = True)  # Add component to the pipeline

## BRAT to BIOES annotation

The next code transform the BRAT annotations into BIOES format. The code return a dict with the token offset and the BIOES schema. In this schema,tokens are annotated using the following tags:


    B: represents a token that conform the begining of an entity.
    I: indicate that the token belongs to an entity.
    O: represents that the token does not belong to an entity.
    E: marks a token as the end of a given entity.
    S: indicates that an entity is comprised of a single token.


In [9]:
############# BIOES NOTATION #####################
BEGIN = 'B'
INSIDE = 'I'
OUTSIDE = 'O'
END = 'E'
SINGLE = 'S'

def getDictEntities(file_ann, ent_classes = ['PROFESION', 'SITUACION_LABORAL']):
  entities = {}
  with open(file_ann) as anns:
    for ann in anns:
      if ann.split('\t')[1].split(' ')[0] in ent_classes:
          ent = ann[:-1].split('\t')[2]
          #print(ent)
          #ent = [token for token in nlp(ent) if not token.is_stop]
          ent = nlp(ent)
          start = int(ann[:-1].split('\t')[1].split(' ')[1])
          end = int(ann[:-1].split('\t')[1].split(' ')[2])
          if (len(ent) == 1):
            entities[(start, end)] = SINGLE + '_' + ann.split('\t')[1].split(' ')[0]
          else:
            entities[(start, start + len(ent[0].text))] = BEGIN + '_' + ann.split('\t')[1].split(' ')[0]
            entities[(end - len(ent[-1].text)), end] = END + '_' + ann.split('\t')[1].split(' ')[0]
            for i in range(len(ent) - 2):
              spaces = (ent[i + 1].idx) - (ent[i].idx + len(ent[i].text))
              start = start + len(ent[i].text) + spaces
              entities[(start, start + len(ent[i + 1].text))] = INSIDE + '_' + ann.split('\t')[1].split(' ')[0]
            
  return entities


In [10]:
import os
import re
#from sklearn.preprocessing import MultiLabelBinarizer

def getElements(sst_home, max_len_seq, getTags = True):
  _words = dict()
  _doc_tags = {}
  _entities = {}
  _docs = {}
  _docs_offset = {}
  #mlb = MultiLabelBinarizer(classes = classes)
    
  for file in [file[:-4] for file in os.listdir(sst_home) if file.endswith('.txt') and not ' ' in file]:
    file_text = os.path.join(sst_home, file + '.txt')
    if getTags:
      file_ann = os.path.join(sst_home, file + '.ann')
      _entities = getDictEntities(file_ann)
    with open(file_text) as f:
      text = f.read()
      spacy_text = nlp(text)
      #spacy_text = [token for token in spacy_text if not token.is_stop]
    
      _tweet = []
      _tweet_tags = []
      _tweet_pos = []
      _tweet_gtags = []
      _tweet_ent = []
      _tweet_offset = []
    
      for token in spacy_text[0:max_len_seq]:
          _tweet.append(token.text)
          _entity = _entities.get((token.idx, token.idx + len(token.text)), 'O')
          _tweet_tags.append(_entity)
          _words[token.text] = _words.get(token.text, 0) + 1
          _tweet_pos.append(token.pos_)
          _tweet_gtags.append(token.tag_)
          _tweet_ent.append(token.ent_iob_ + '_' + token.ent_type_)
          _tweet_offset.append((token.text, token.idx, token.idx + len(token.text)))

    _docs[file] = (_tweet, _tweet_pos, _tweet_gtags, _tweet_ent)
    _doc_tags[file] = _tweet_tags
    _docs_offset[file] = _tweet_offset

  #_words = list(_words)

  return _words, _docs, _doc_tags, _docs_offset

## Getting the 2idx

Our model is based in a Bag of Words/Charachters/Syllables/PoS-Tags, so we need to create a dictionary to assign each of one see it during training to a cardinal id.

In [11]:
def get2idx(words, chars, pos, gramm_tags, sylls, ents):
    
  ET = ['B_SITUACION_LABORAL', 'B_PROFESION', 
        'I_SITUACION_LABORAL', 'I_PROFESION', 
        'E_SITUACION_LABORAL', 'E_PROFESION', 
        'S_SITUACION_LABORAL', 'S_PROFESION',
        'O']

  word2idx = {w: i + 2 for i, w in enumerate(words)}
  word2idx["#ENDPAD"] = 0
  word2idx["UNK"] = 1

  char2idx = {char:i + 2 for i,char in enumerate(chars)}
  char2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  char2idx["UNK"] = 1

  pos2idx = {pos:i + 2 for i,pos in enumerate(pos)}
  pos2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  pos2idx["UNK"] = 1

  grammtags2idx = {tag:i + 2 for i,tag in enumerate(gramm_tags)}
  grammtags2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  grammtags2idx["UNK"] = 1

  sylls2idx = {syll:i + 2 for i,syll in enumerate(sylls)}
  sylls2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  sylls2idx["UNK"] = 1

  tag2idx = {tag:i + 1 for i,tag in enumerate(ET)}
  tag2idx["#ENDPAD"] = 0 

  ent2idx = {ent:i + 2 for i,ent in enumerate(ents)}
  ent2idx["#ENDPAD"] = 0  # to ignore this by mask_zero = True
  ent2idx["UNK"] = 1

  return word2idx, char2idx, tag2idx, pos2idx, grammtags2idx, sylls2idx, ent2idx

## Getting the input

In [12]:
### CHARACTERS ###

def getCharacterInput(sentences, max_seq_len, max_chars_len, char2idx):
  X_char = []
  for sentence in sentences:
    sent_seq = []
    # max_len = 75
    for i in range(max_seq_len):
      word_seq = []
      # char sequence for words
      for j in range(max_chars_len):
        try:
          # chars of specific sentence of i
          word_seq.append(char2idx.get(sentence[i][j], 1)) 
        except:  # if char-sequence is out of range , pad it with "PAD" tag
          word_seq.append(char2idx.get("#ENDPAD"))

      sent_seq.append(word_seq)
    # append sentence sequences as character-by-character to X_char for Model input
    X_char.append(np.array(sent_seq))

  return np.array(X_char)

### SYLLABLES ###

def getSyllsInput(sentences, max_seq_len, max_sylls_len, sylls2idx):
  X_syll = []
  for sentence in sentences:
    sent_seq = []
    # max_len = 75
    for i in range(max_seq_len):
      word_seq = []
      syllables = []
      try:
        syllables = silabizer(sentence[i])
      except:
        pass
      for j in range(max_sylls_len):
        try:
          # chars of specific sentence of i
          word_seq.append(sylls2idx.get(str(syllables[j]).lower(), 1)) 
        except:  # if char-sequence is out of range , pad it with "PAD" tag
          word_seq.append(sylls2idx.get("#ENDPAD"))

      sent_seq.append(word_seq)
    # append sentence sequences as character-by-character to X_char for Model input
    X_syll.append(np.array(sent_seq))

  return np.array(X_syll)

from keras.preprocessing.sequence import pad_sequences

### WORDS ###

def getWordInput(sentences, max_seq_len, word2idx):
  X = [[word2idx.get(w,1) for w in s[:max_seq_len]] for s in sentences]
  X = pad_sequences(maxlen = max_seq_len, sequences = X, truncating= 'post', padding ='post', value=0 )
  return np.array(X)

### PoS-Tags ###

def getPosInput(sentences, max_seq_len, pos2idx):
  X_pos = [[pos2idx.get(w,1) for w in s] for s in sentences]
  X_pos = pad_sequences(maxlen = max_seq_len, sequences = X_pos, truncating= 'post', padding ='post', value=0 )
  return np.array(X_pos)

def getGTagInput(sentences, max_seq_len, grammtags2idx):
  X_gramm_tags = [[grammtags2idx.get(w,1) for w in s] for s in sentences]
  X_gramm_tags = pad_sequences(maxlen = max_seq_len, sequences = X_gramm_tags, truncating= 'post', padding ='post', value=0 )
  return np.array(X_gramm_tags)

def getEntInput(sentences, max_seq_len, ent2idx):
  X_ents = [[ent2idx.get(w,1) for w in s] for s in sentences]
  X_ents = pad_sequences(maxlen = max_seq_len, sequences = X_ents, truncating= 'post', padding ='post', value=0 )
  return np.array(X_ents)

from keras.utils.np_utils import to_categorical

### NER TAG ###
def getY(tags, max_seq_len, tag2idx):
  y = [[tag2idx[t] for t in sent_tags] for sent_tags in tags]
  y = pad_sequences(maxlen = max_seq_len, sequences = y, padding = "post", truncating = "post", value = tag2idx["#ENDPAD"])
  
  y = [to_categorical(i, num_classes = len(tag2idx)) for i in y]
  return np.array(y)

Using TensorFlow backend.


## Getting the total words

Cause we are using pre-trained embeddings, we use as our vocabulary the words contained in both the training, development and test sets

In [12]:
_total_words = set()
sst_home_train = sst_home + '/profner-data/subtask-2/brat/train'
for file in [file[:-4] for file in os.listdir(sst_home_train) if file.endswith('.txt') and not ' ' in file]:
    file_text = os.path.join(sst_home_train, file + '.txt')
    
    with open(file_text) as f:
        text = f.read()
        doc = nlp(text)
        for token in doc:
            _total_words.add(token.text)
            #if not token.is_stop:
                #_total_words.add(token.text)
             
sst_home_test = sst_home + '/profner-data/subtask-2/brat/valid'
for file in [file[:-4] for file in os.listdir(sst_home_test) if file.endswith('.txt') and not ' ' in file]:
    file_text = os.path.join(sst_home_test, file + '.txt')
    
    with open(file_text) as f:
        text = f.read()
        doc = nlp(text)
        for token in doc:
            _total_words.add(token.text)
            #if not token.is_stop:
                #_total_words.add(token.text)

'''
    //TODO: HASTA QUE LIBEREN EL TEST SET, ESTO DEBE PERMANECER COMENTADO
    
sst_home_test = sst_home + 'test-background-set-to-publish/'
for file in [file[:-4] for file in os.listdir(sst_home_test) if file.endswith('.txt') and not ' ' in file]:
    file_text = os.path.join(sst_home_test, file + '.txt')
    
    with open(file_text) as f:
        text = f.read()
        doc = nlp(text)
        for token in doc:
            if not token.is_stop:
                _total_words.add(token.text)
'''

_words = list(_total_words)

##  Generating Training Data

The next code generate the training data

In [13]:
import numpy as np

sst_home_train = sst_home + '/profner-data/subtask-2/brat/train'

MAX_CHARS_LEN = 25
MAX_SEQ_LEN = 75
MAX_SYLLS_LEN = 10

silabizer = silabizer()

words_tr, docs, docs_tags, _ = getElements(sst_home_train, MAX_SEQ_LEN)
words_tr = list(words_tr.keys())
chars = list(set(''.join(words_tr)))
sylls = list(set(str(syll).lower() for word in words_tr for syll in silabizer(word)))

tweets_train = [tweet[0] for tweet in docs.values()]
tweets_pos = [tweet[1] for tweet in docs.values()]
tweets_gtags = [tweet[2] for tweet in docs.values()]
tweets_ents = [tweet[3] for tweet in docs.values()]

all_pos = set(pos for sent in tweets_pos for pos in sent)
all_gtags = set(gtag for sent in tweets_gtags for gtag in sent)
all_ents = set(ent for sent in tweets_ents for ent in sent)

tags = [tags for tags in docs_tags.values()]

word2idx, char2idx, tag2idx, pos2idx, grammtags2idx, sylls2idx, ent2idx = get2idx(_words, chars, all_pos, all_gtags, sylls, all_ents)

X_char_train = getCharacterInput(tweets_train, MAX_SEQ_LEN, MAX_CHARS_LEN, char2idx)
X_words_train = getWordInput(tweets_train, MAX_SEQ_LEN, word2idx)
X_pos_train = getPosInput(tweets_pos, MAX_SEQ_LEN, pos2idx)
X_tag_train = getGTagInput(tweets_gtags, MAX_SEQ_LEN, grammtags2idx)
X_syll_train = getSyllsInput(tweets_train, MAX_SEQ_LEN, MAX_SYLLS_LEN, sylls2idx)
X_ent_train = getEntInput(tweets_ents, MAX_SEQ_LEN, ent2idx)

y_train = getY(tags, MAX_SEQ_LEN, tag2idx)

In [17]:
import pickle5 as pickle
import numpy as np

train_bert = {}
with open(sst_home + '/saved_data/bert_train.pickle', 'rb') as handle:
    train_bert = pickle.load(handle)

X_train_bert = []
sst_home_train = sst_home + '/profner-data/subtask-2/brat/train'
for file in [file[:-4] for file in os.listdir(sst_home_train) if file.endswith('.txt') and not ' ' in file]:
    bert_vector = train_bert[file]
    #bert_vector = pad_sequences(maxlen = 100, sequences = bert_vector, truncating= 'post', padding ='post', value=np.zeros(768))
    X_train_bert.append(bert_vector)
X_train_bert = pad_sequences(maxlen = 75, sequences = X_train_bert, truncating= 'post', padding ='post', value=np.zeros(1536))
del train_bert

## Generating Validation Data

The next code generate the validation data.

In [14]:
import numpy as np

sst_home_test = sst_home + '/profner-data/subtask-2/brat/valid'

_, docs, docs_tags, docs_offset = getElements(sst_home_test, MAX_SEQ_LEN)

tweets_test = [tweet[0] for tweet in docs.values()]
tweets_pos = [tweet[1] for tweet in docs.values()]
tweets_gtags = [tweet[2] for tweet in docs.values()]
tweets_ents = [tweet[3] for tweet in docs.values()]


tags_test = [tags for tags in docs_tags.values()]


X_char_test = getCharacterInput(tweets_test, MAX_SEQ_LEN, MAX_CHARS_LEN, char2idx)
X_words_test = getWordInput(tweets_test, MAX_SEQ_LEN, word2idx)
X_pos_test = getPosInput(tweets_pos, MAX_SEQ_LEN, pos2idx)
X_tag_test = getGTagInput(tweets_gtags, MAX_SEQ_LEN, grammtags2idx)
X_syll_test = getSyllsInput(tweets_test, MAX_SEQ_LEN, MAX_SYLLS_LEN, sylls2idx)
X_ent_test = getEntInput(tweets_ents, MAX_SEQ_LEN, ent2idx)

y_test = getY(tags_test, MAX_SEQ_LEN, tag2idx)

In [13]:
import pickle5 as pickle
import numpy as np
import os

test_bert = {}
with open(sst_home + '/saved_data/bert_valid.pickle', 'rb') as handle:
    test_bert = pickle.load(handle)
    
def getBertInput(file_name):
    bert_vector = test_bert[file_name]
    #bert_vector = pad_sequences(maxlen = 100, sequences = bert_vector, truncating= 'post', padding ='post', value=np.zeros(768))
    X_test_bert = [bert_vector]
    X_test_bert = pad_sequences(maxlen = 75, sequences = X_test_bert, truncating= 'post', padding ='post', value=np.zeros(1536))
    return X_test_bert

X_test_bert = []
sst_home_train = sst_home + '/profner-data/subtask-2/brat/valid'
for file in [file[:-4] for file in os.listdir(sst_home_train) if file.endswith('.txt') and not ' ' in file]:
    bert_vector = test_bert[file]
    #bert_vector = pad_sequences(maxlen = 100, sequences = bert_vector, truncating= 'post', padding ='post', value=np.zeros(768))
    X_test_bert.append(bert_vector)
X_test_bert = pad_sequences(maxlen = 75, sequences = X_test_bert, truncating= 'post', padding ='post', value=np.zeros(1536))
del test_bert

## Word embeddings from Spanish Medical Corpora & Twitter

In this model we'll use two different pre-trained word embeddings:

1. The Word Embeddings from Spanish Medical Corpora can be found in https://www.aclweb.org/anthology/W19-1916/
2. The Word Embedding from Spanish Twitter (Covid-19) can be found in https://zenodo.org/record/4449930#.YBbYOtaCE5k

In [14]:
import fasttext
import fasttext.util

def getEmbeddingMatrix(words2idx, emb_dim, model):
  embedding_matrix = np.zeros((len(words2idx), emb_dim))
  for word, i in words2idx.items():
    embedding_matrix[i] = model[word]

  return embedding_matrix

In [16]:
sst_home_embeddings = sst_home + '/fast-text-model/'

### SPANISH MEDICAL CORPORA ###
ft = fasttext.load_model(sst_home_embeddings + 'cantemist-resource.bin')
medical_embedding_matrix = getEmbeddingMatrix(word2idx, ft.get_dimension(), ft)
del ft

### SPANISH TWITTER COVID-19 ###
ft = fasttext.load_model(sst_home_embeddings + 'covid_19_es_twitter_cbow_cased.bin')
twitter_embedding_matrix = getEmbeddingMatrix(word2idx, ft.get_dimension(), ft)
del ft

spacy_embedding_matrix = np.array([nlp(word)[0].vector for word in word2idx.keys()])


## Getting Entities to construct Cosine Similarity Vectors

For each word in our vocabulary, we generate a vector with the cosine similarity of all entities found in training data. To help with the computational cost, we parallelize the proccess using the *multiprocessing* library

1. Extracting the different entities
2. Defining the cosine similarity function
3. Parallelization of the vector obtention proccess

In [20]:
import os
entities = {}
sst_home_train = sst_home + '/profner-data/subtask-2/brat/train'
for file in [file[:-4] for file in os.listdir(sst_home_train) if file.endswith('.ann') and not ' ' in file]:
    file_text = os.path.join(sst_home_train, file + '.ann')
    with open(file_text) as f:
        for line in f.readlines():
            ent_type = line.split('\t')[1].split(' ')[0]
            ent_form = line.split('\t')[-1][:-1]
            
            entities[ent_type] = entities.get(ent_type, []) + [ent_form]
            
profesion_entities = list(set(entities['PROFESION']))
sit_lab_entities = list(set(entities['SITUACION_LABORAL']))

entities = profesion_entities + sit_lab_entities
dim = len(entities)

In [21]:
from numpy import dot
from numpy.linalg import norm

cos_sim = lambda a,b: dot(a, b)/(norm(a)*norm(b))

In [22]:
from multiprocessing import Pool
import numpy as np

sst_home_embeddings = sst_home + '/fast-text-model/'

ft = fasttext.load_model(sst_home_embeddings + 'covid_19_es_twitter_cbow_cased.bin')
entities_vectors_I = {}
for i, entity in enumerate(entities):
    entities_vectors_I[entity] = np.mean([ft[token.text] for token in nlp(entity)], axis = 0)    
del ft

ft = fasttext.load_model(sst_home_embeddings + 'cantemist-resource.bin')

entities_vectors_II = {}
for i, entity in enumerate(entities):
    entities_vectors_II[entity] = np.mean([ft[token.text] for token in nlp(entity)], axis = 0)    
del ft


def getVector(t):
    word = t[0]
    key = t[1]
    vector = np.zeros(dim * 2)
    for i, entity in enumerate(entities):
        vector[i] = cos_sim(twitter_embedding_matrix[key], entities_vectors_I[entity])
        vector[i + dim] = cos_sim(medical_embedding_matrix[key], entities_vectors_II[entity])
    
    return vector
with Pool(12) as p:
     cosine_matrix = p.map(getVector, list(word2idx.items()))
        
cosine_matrix = np.array(cosine_matrix)

## Saving Data

We save the generated data so that it is not necessary to generate it again next time.

In [23]:
import pickle

train_data = [X_char_train, X_words_train, X_pos_train, X_tag_train, X_syll_train, X_ent_train, X_train_bert, y_train]
with open(sst_home + '/saved_data/train.pickle', 'wb') as handle:
    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

test_data = [X_char_test, X_words_test, X_pos_test, X_tag_test, X_syll_test, X_ent_test, X_test_bert, y_test]
with open(sst_home + '/saved_data/test.pickle', 'wb') as handle:
    pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

embedding_matrix = [medical_embedding_matrix, twitter_embedding_matrix, cosine_matrix, spacy_embedding_matrix]   
with open(sst_home + '/saved_data/we.pickle', 'wb') as handle:
    pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

two2idx = [word2idx, char2idx, tag2idx, pos2idx, grammtags2idx, sylls2idx, ent2idx]
with open(sst_home + '/saved_data/2idx.pickle', 'wb') as handle:
    pickle.dump(two2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)



## Loading Data

In case the data is previously saved, we load these.

In [15]:
import pickle

X_train = []
with open(sst_home + '/saved_data/train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)

X_test = []
with open(sst_home + '/saved_data/test.pickle', 'rb') as handle:
    X_test = pickle.load(handle)

embedding_matrixes = []
with open(sst_home + '/saved_data/we.pickle', 'rb') as handle:
    embedding_matrix = pickle.load(handle)

two2idx = []
with open(sst_home + '/saved_data/2idx.pickle', 'rb') as handle:
    two2idx = pickle.load(handle)

X_char_train, X_words_train, X_pos_train, X_tag_train, X_syll_train, X_ent_train, X_train_bert, y_train = X_train
X_char_test, X_words_test, X_pos_test, X_tag_test, X_syll_test, X_ent_test, X_test_bert, y_test = X_test
word2idx, char2idx, tag2idx, pos2idx, grammtags2idx, sylls2idx, ent2idx = two2idx
medical_embedding_matrix, twitter_embedding_matrix, cosine_matrix, spacy_embedding_matrix = embedding_matrix

## Deep Learning Architecture

In [16]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-k5oa8nwk
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-k5oa8nwk
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101064 sha256=9f3a63ad58f3ea253d00ac68a08fa59468152251c8f2669c3a9788ae047e42e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-pcw3u071/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [17]:
import tensorflow as tf
from keras.models import Model
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Concatenate, Input, SpatialDropout1D
from keras.layers import Conv1D, MaxPooling1D,Flatten,GlobalMaxPooling1D, Reshape, RepeatVector,  Dot, GRU, Activation
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint

CHAR_EMBEDDINGS_SIZE = 30    # Characters Embeddings Size
SYLL_EMBEDDINGS_SIZE = 75    # Syllable Embeddings Size
WORD_EMBEDDINGS_SIZE = 300   # Word Embeddings Size
COSINE_EMBEDDING_SIZE = cosine_matrix.shape[1]
POS_EMBEDDING_SIZE = 20      # PoS Embedding Size
GTAGS_EMBEDDING_SIZE = 20    # Tag Embedding Size

MAX_CHARS_LEN = 25           # Max sequence char length
MAX_SEQ_LEN = 75             # Max sequence word length Level II
MAX_SYLLS_LEN = 10           # Max sequence sylls length

CONV_FILTERS = 50            # Convolutional Filters in Character and Syllable Convolutional Layer
LSTM_UNITS = 300             # LSTM Units in both LSTM layers
DENSE_UNITS = 200            # Number of units in Dense layer  

BERT_DIM = 1536

word_input = Input(shape=(MAX_SEQ_LEN,), name = 'word_input')
pos_input = Input(shape=(MAX_SEQ_LEN,), name = 'pos_input')
gtag_input = Input(shape=(MAX_SEQ_LEN,), name = 'tag_input')
char_input = Input(shape=(MAX_SEQ_LEN, MAX_CHARS_LEN), name = 'char_input')
sylls_input = Input(shape=(MAX_SEQ_LEN, MAX_SYLLS_LEN), name = 'sylls_input')
ents_input = Input(shape=(MAX_SEQ_LEN,), name = 'ents_input')
bert_input = Input(shape=(MAX_SEQ_LEN, BERT_DIM,), name = 'bert_input')

###################### CHARACTER SEQUENCE PROCCESSED BY CONVOLUTIONAL LAYER ################################
char_embedding = TimeDistributed(Embedding(input_dim = len(char2idx), output_dim = CHAR_EMBEDDINGS_SIZE, input_length=MAX_CHARS_LEN, name = 'char_embeddings', trainable = True))(char_input)
conv_1d = TimeDistributed(Conv1D(filters = CONV_FILTERS, kernel_size = 3,padding = "valid", activation = "relu", name="Conv1D_char"))(char_embedding)
conv_1d = TimeDistributed(Dropout(0.4))(conv_1d)
maxpool1d = TimeDistributed(GlobalMaxPooling1D(), name = 'max_pooling')(conv_1d)
char_enc = TimeDistributed(Flatten(), name = 'char_enc')(maxpool1d)

###################### SYLLABLE SEQUENCE PROCCESSED BY CONVOLUTIONAL LAYER ################################
syll_embedding = TimeDistributed(Embedding(input_dim = len(sylls2idx), output_dim = SYLL_EMBEDDINGS_SIZE, input_length=MAX_SYLLS_LEN, name = 'sylls_embeddings', trainable = True))(sylls_input)
conv_1d_syll = TimeDistributed(Conv1D(filters = CONV_FILTERS, kernel_size = 3,padding="valid", activation="relu", name="Conv1D_syll"))(syll_embedding)
conv_1d_syll = TimeDistributed(Dropout(0.4))(conv_1d_syll)
maxpool1d_syll = TimeDistributed(GlobalMaxPooling1D())(conv_1d_syll)
syll_enc = TimeDistributed(Flatten())(maxpool1d_syll)

##################### PoS + TAG EMBEDDINGS ################################################################
pos_embedding = Embedding(input_dim = len(pos2idx), output_dim = POS_EMBEDDING_SIZE, input_length = MAX_SEQ_LEN, name = 'pos_embeddings', trainable = True)(pos_input)
gtags_embedding = Embedding(input_dim = len(grammtags2idx), output_dim = GTAGS_EMBEDDING_SIZE, input_length = MAX_SEQ_LEN, name = 'gtags_embeddings', trainable = True)(gtag_input)
ents_embeddings =  Embedding(input_dim = len(ent2idx), output_dim = 20, input_length = MAX_SEQ_LEN, name = 'ents_embeddings', trainable = True)(ents_input)
##################### WORD EMBEDDING LAYER ################################################################
medical_embedding_layer = Embedding(input_dim = len(word2idx), output_dim = WORD_EMBEDDINGS_SIZE, weights=[medical_embedding_matrix], trainable=True, name = 'medical_word_embeddings')
medical_word_embedding = medical_embedding_layer(word_input)

twitter_embedding_layer = Embedding(input_dim = len(word2idx), output_dim = WORD_EMBEDDINGS_SIZE, weights=[twitter_embedding_matrix], trainable=True, name = 'twitter_word_embeddings')
twitter_word_embedding = twitter_embedding_layer(word_input)

cosine_embedding_layer = Embedding(input_dim = len(word2idx), output_dim = COSINE_EMBEDDING_SIZE, weights=[cosine_matrix], trainable=True, name = 'cosine_distance_embeddings')
cosine_word_embedding = cosine_embedding_layer(word_input)

#spacy_embedding_layer = Embedding(input_dim = len(word2idx), output_dim = WORD_EMBEDDINGS_SIZE, weights=[spacy_embedding_matrix], trainable=True, name = 'spacy_embeddings')
#spacy_word_embedding = spacy_embedding_layer(word_input)
################### CONCATENATE INPUT FEATURES ###########################################
x = Concatenate(axis = -1)([medical_word_embedding, twitter_word_embedding, bert_input, pos_embedding, gtags_embedding, ents_embeddings, char_enc, syll_enc])

################## BiLSTM MAX SEQUENCE WORD LENGTH 50 ######################################################
encoder_outputs, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(units=LSTM_UNITS//2, return_sequences=True,recurrent_dropout=0, return_state = True, recurrent_activation = 'sigmoid'))(x)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

#encoder_outputs = Dropout(0.4)(encoder_outputs)
#encoder_states = Dropout(0.4)(encoder_states)

decoder_outputs, _, _ = LSTM(units=LSTM_UNITS, recurrent_dropout=0, return_sequences = True, recurrent_activation = 'sigmoid', return_state = True)(x, initial_state = encoder_states)
attention = Dot(axes = (2,2))([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)
context = Dot(axes=(2,1))([attention, encoder_outputs])
x = Concatenate()([context, decoder_outputs, cosine_word_embedding])
x = Dropout(0.4)(x)

################## DENSE LAYER ############################################################################
x = Dense(DENSE_UNITS, activation='tanh')(x)

################## CRF LAYER ####################################################################################
crf = CRF(len(tag2idx), sparse_target = False)
loss = crf.loss_function
y_output = crf(x)

model = Model(inputs = [char_input, word_input, pos_input, gtag_input, sylls_input, ents_input, bert_input], outputs = y_output)
model.compile(optimizer = "adam", loss = loss, metrics = [crf.accuracy])
#model.load_weights(sst_home + '/model_weights/best_model.hdf5')
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/sergio/anaconda3/envs/tfm/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/sergio/anaconda3/envs/tfm/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 75, 25)       0                                            
__________________________________________________________________________________________________
sylls_input (InputLayer)        (None, 75, 10)       0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 75, 25, 30)   17760       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_4 (TimeDistrib (None, 75, 10, 75)   990900      sylls_input[0][0]                
__________________________________________________________________________________________________
time_distr

In [21]:
NUM_EPOCHS = 1
checkpoint = ModelCheckpoint(sst_home + '/model_weights/best_model.hdf5', monitor='loss', verbose=1, save_best_only=True, mode='auto', period=1)
history = model.fit([X_char_train, X_words_train, X_pos_train, X_tag_train, X_syll_train, X_ent_train, X_train_bert], y_train,
                    validation_data = ([X_char_test, X_words_test, X_pos_test, X_tag_test, X_syll_test, X_ent_test, X_test_bert], y_test),
                    batch_size = 32,
                    epochs = NUM_EPOCHS,
                    callbacks=[checkpoint])

Train on 6000 samples, validate on 2000 samples
Epoch 1/1
6000/6000 [==============================] - 107s 18ms/step - loss: 0.0023 - crf_viterbi_accuracy: 0.9993 - val_loss: 0.0078 - val_crf_viterbi_accuracy: 0.9979

Epoch 00001: loss improved from inf to 0.00231, saving model to /home/sergio/Escritorio/ProfNER/model_weights/best_model.hdf5


## Generating the annotations (7b)

The next step is generate the annotations in the right format to be evaluated

In [ ]:
def annotate(y_pred, offset, file_name, path_to_save):
    SINGLE = 'S'
    BEGIN = 'B'
    END = 'E'
    INSIDE = 'I'
    #i = 1
    has_begin = False
    current_ent = None
    text = ''
  #file = path_to_save + file_name + '.ann'
    for ann, info in zip(y_pred, offset):
      #if not((ann[0] == 'O' and current_ent != None) or (current_ent != None and current_ent != ann[2:])):
      if ann[0] == SINGLE:
        if has_begin:
          print('ERROR: Llego una SINGLE sin terminar la anterior BI -> S')
          print('{}\t{}\t{}\t{}\t{}\n'.format(file_name, start, end, ann[2:], entity.replace('\n', '#SALTO')))
          print()
          text = text + '{}\t{}\t{}\t{}\t{}\n'.format(file_name, start, end, ann[2:], entity.replace('\n', '#SALTO'))
          current_ent = None
          has_begin = False

        text = text + '{}\t{}\t{}\t{}\t{}\n'.format(file_name, info[1], info[2], ann[2:], info[0].replace('\n', '#SALTO'))
        current_ent = None

      if ann[0] == BEGIN:
        if has_begin:
          print('ERROR: Llego un BEGIN sin terminar la anterior BI -> B')
          print('{}\t{}\t{}\t{}\t{}\n'.format(file_name, start, end, ann[2:], entity.replace('\n', '#SALTO')))
          print()
          text = text + '{}\t{}\t{}\t{}\t{}\n'.format(file_name, start, end, ann[2:], entity.replace('\n', '#SALTO'))
          #i = i + 1

        entity = info[0]
        start = info[1]
        end = info[2]
        current_ent = ann[2:]
        has_begin = True

      if ann[0] == INSIDE:
        current_ent = ann[2:]
        if not has_begin:
          print('ERROR: Llego INSIDE sin terminar la anterior BI -> I')
          print()
          entity = info[0]
          start = info[1]
          end = info[2]
          has_begin = True

        else:
          entity = entity + ' ' + info[0]
          end = info[2]

      if ann[0] == END:
        current_ent = None
        if not has_begin:
          print('ERROR: Llego un END sin tener BEGIN')
          print('{}\t{}\t{}\t{}\t{}\n'.format(file_name, info[1], info[2], ann[2:], info[0].replace('\n', '#SALTO')))
          print()
          #text = text + '{}\t{}\t{}\t{}\t{}\n'.format(file_name, info[1], info[2], ann[2:], info[0].replace('\n', '#SALTO'))
          #i = i + 1
        else:
          entity_text = entity + ' ' + info[0]
          text = text + '{}\t{}\t{}\t{}\t{}\n'.format(file_name, start, info[2], ann[2:], entity_text.replace('\n', '#SALTO'))
          has_begin = False
          #i = i + 1
      '''else:
          print('ERROR: Cambio de entidad sin final')
          #print(f'actual: {current_ent}, siguiente {ann[2:]}')
          print('{}\t{}\t{}\t{}\t{}\n'.format(file_name, start, end, current_ent, entity.replace('\n', '#SALTO')))
          print()
          text = text + '{}\t{}\t{}\t{}\t{}\n'.format(file_name, start, end, current_ent, entity.replace('\n', '#SALTO'))
          current_ent = None
          has_begin = False '''
          

    return text

In [22]:
SINGLE = 'S'
BEGIN = 'B'
END = 'E'
INSIDE = 'I'
OUT = 'O'

def annotateV2(y_pred, offset, file_name, extra = None):    
    max_index = len(y_pred)
    if '#ENDPAD' in y_pred:
        max_index = y_pred.index('#ENDPAD')
        
    right_moves = {'B': [ 'I', 'E'],
                   'I': ['I', 'E'],
                   'E': ['B', 'S', 'O'],
                   'S': ['B', 'S', 'O'],
                   'O': ['B', 'S', 'O']}
    text = ''
    entity = ''
    start = -1
    end = -1
    
    has_error = False
    for i in range(max_index - 1):
        if not (has_error and y_pred[i][0] in ['I','E']):
            has_error = False
            ann = y_pred[i][0]
            info = offset[i]
            next_ann = y_pred[i + 1][0]
            entity_class = y_pred[i][2:]
            ### SI LA EL MOVIMIENTO ES PERMITIDO, SE EJECUTA SU POLITICA NORMAL
            if next_ann in right_moves[ann]:
                info = offset[i]
                if ann == BEGIN:
                    entity = info[0]
                    start = info[1]
                    end = info[2]
                if ann == INSIDE:
                    entity = entity + ' ' + info[0]
                    end = info[2]
                if ann == END:
                    entity = entity + ' ' + info[0]
                    end = info[2]
                    text = text + f'{file_name}\t{start}\t{end}\t{entity_class}\t{entity}\n'
                    entity = ''
                    start = -1
                    end = -1
                if ann == SINGLE:
                    text = text + f'{file_name}\t{info[1]}\t{info[2]}\t{entity_class}\t{info[0]}\n'
                    entity = ''
                    start = -1
                    end = -1
                if ann == OUT:
                    entity = ''
                    start = -1
                    end = -1
            else:
                has_error = True
                
                file_ann = sst_home + f'/profner-data/subtask-2/brat/valid/{file_name}.ann' 
                with open(file_ann) as file:
                  #print(file_ann)
                  print(file.read())
                
                '''
                if ann == 'I' and next_ann == 'O':
                    entity = " ".join([entity, info[0], offset[i + 1][0]])
                    print(f'{file_name}\t{start}\t{offset[i + 1][2]}\t{entity_class}\t{entity}\n')
                    text = text + f'{file_name}\t{start}\t{offset[i + 1][2]}\t{entity_class}\t{entity}\n'
                
                if ann == 'S' and next_ann == 'I' and y_pred[i + 2][0] == 'E':
                    entity = " ".join([info[0], offset[i + 1][0], offset[i + 2][0]])
                    #print(f'{file_name}\t{info[1]}\t{offset[i + 2][2]}\t{entity_class}\t{entity}\n')
                    text = text + f'{file_name}\t{info[1]}\t{offset[i + 2][2]}\t{entity_class}\t{entity}\n'
                    
                if ann == 'S' and next_ann == 'I' and y_pred[i + 2][0] == 'O':
                    entity = " ".join([info[0], offset[i + 1][0]])
                    #print(f'{file_name}\t{info[1]}\t{offset[i + 1][2]}\t{entity_class}\t{entity}\n')
                    text = text + f'{file_name}\t{info[1]}\t{offset[i + 1][2]}\t{entity_class}\t{entity}\n'
   
                if ann == 'E' and next_ann == 'I':
                    entity = " ".join([entity, info[0]])
                    print(f'{file_name}\t{start}\t{offset[i][2]}\t{entity_class}\t{entity}\n')
                    text = text + f'{file_name}\t{start}\t{offset[i][2]}\t{entity_class}\t{entity}\n'
                    
                if ann == 'E' and next_ann == 'E':
                    entity = " ".join([entity, info[0], offset[i + 1][0]])
                    print(f'{file_name}\t{start}\t{offset[i + 1][2]}\t{entity_class}\t{entity}\n')
                    text = text + f'{file_name}\t{start}\t{offset[i + 1][2]}\t{entity_class}\t{entity}\n'
                '''
                if ann == 'E' and next_ann == 'E':
                    print('###### ERROR #######')
                    print(f'Error: {y_pred[i - 1]} -> {ann} -> {next_ann} -> {y_pred[i + 2]}')
                    info = offset[i]
                    if entity == '':
                        print(f'Entidad: "{info[0]}" Clase: "{y_pred[i][2:]}"')
                    else:
                        print(f'Entidad: {entity  + info[0]} Offset: ({start},{info[2]})')
                    if next_ann != OUT:
                        pass
                    print(f'Anterior Entidad {offset[i - 1][0]}')
                    print(f'Siguiente Entidad {offset[i + 1][0]}')
                    print()
                    
                entity = ''
                start = -1
                end = -1
            
    return text  

In [23]:
import numpy as np

sst_home_test = sst_home + '/profner-data/subtask-2/brat/valid'
idx2tag = {idx:tag for (tag, idx) in tag2idx.items()}

_, docs, _, docs_offset = getElements(sst_home_test, MAX_SEQ_LEN, getTags = False)

result_path = sst_home + '/final-result/'
text = ''

for (file_name, doc_sents) in docs.items():
  X_char = getCharacterInput([docs[file_name][0]], MAX_SEQ_LEN, MAX_CHARS_LEN, char2idx)
  X_words = getWordInput([docs[file_name][0]], MAX_SEQ_LEN, word2idx)
  X_pos = getPosInput([docs[file_name][1]], MAX_SEQ_LEN, pos2idx)
  X_tag = getGTagInput([docs[file_name][2]], MAX_SEQ_LEN, grammtags2idx)
  X_syll = getSyllsInput([docs[file_name][0]], MAX_SEQ_LEN, MAX_SYLLS_LEN, sylls2idx)
  X_ent = getEntInput([docs[file_name][2]], MAX_SEQ_LEN, ent2idx)
  X_bert = getBertInput(file_name)

  y_pred = model.predict([X_char, X_words, X_pos, X_tag, X_syll, X_ent, X_bert])
  y_pred = [list(map(lambda x: idx2tag[np.argmax(x)], sent)) for sent in y_pred][0]
  offset_test = docs_offset[file_name]
  text = text + annotateV2(y_pred, offset_test, file_name, result_path)

text = 'tweet_id\tbegin\tend\ttype\textraction\n' + text
with open(result_path + 'results.tsv', 'w') as f:
    f.write(text)

T1	PROFESION 130 166	Directora de Personas y Comunicación

T1	PROFESION 45 65	pastores evangélicos
T2	PROFESION 3 16	Guardia Civil


T1	PROFESION 172 182	limpiadora
T2	PROFESION 24 49	limpiadora de un hospital
T3	PROFESION 118 130	altos cargos

T1	PROFESION 172 182	limpiadora
T2	PROFESION 24 49	limpiadora de un hospital
T3	PROFESION 118 130	altos cargos

T1	PROFESION 15 48	vicepresidente d asuntos sociales
T2	PROFESION 176 190	vicepresidente


T1	PROFESION 8 18	presidente
T2	SITUACION_LABORAL 95 107	trabajadores


T1	PROFESION 14 45	Director General de la @policia

T1	PROFESION 14 45	Director General de la @policia

T1	PROFESION 3 23	presidente de la Fed


T1	PROFESION 5 28	ministra de @Defensagob

T1	PROFESION 151 177	profesor de la Universidad
T2	PROFESION 126 138	epidemiólogo

T1	PROFESION 101 109	director

T1	SITUACION_LABORAL 34 46	trabajadores
T2	SITUACION_LABORAL 74 83	empleados
T3	SITUACION_LABORAL 179 188	empleados

T1	PROFESION 9 31	subsecretario de salud

T1	PROFESION 69 88	

## Generating the annotations (7a)

In [ ]:
print(text)

In [ ]:
!pip install google_trans_new
!pip install PyDictionary

In [ ]:
for token in nlp('El capellán'):
    print(token.pos_.title())

In [ ]:
translate_text = translator.translate('Él',lang_src='es',lang_tgt='en')
print(translate_text)
try:
    a = dictionary.meaning(translate_text)
except Exception as e:
    pass

In [ ]:
from google_trans_new import google_translator  
from PyDictionary import PyDictionary

translator = google_translator() 
dictionary = PyDictionary()
def getSpanishMeaning(word):
    translate_text = translator.translate(word,lang_src='es',lang_tgt='en')
    english_meaning = dictionary.meaning(translate_text)
    spanish_meanings = {}
    if english_meaning:
        for tag in english_meaning.keys():
            for mean in english_meaning[tag]:
                translate_text = translator.translate(mean,lang_src='en',lang_tgt='es')
                if not type(translate_text) == str:
                    translate_text = translate_text[0]
                translate_text = translate_text.strip().replace('(', '')
                spanish_meanings[tag.upper()] = spanish_meanings.get(tag.upper(), []) + [translate_text]
    return spanish_meanings

getSpanishMeaning('papá')

In [ ]:
users = {user for user in word2idx.keys() if user.startswith('@')}
hashtags = {user for user in word2idx.keys() if user.startswith('#')}

In [ ]:
hashtags

In [ ]:
len(set(map(str.lower, users)))

In [ ]:
import re

pattern = re.compile(r"[A-Z][a-z]+|\d+|[A-Z]+(?![a-z])")
def split_hashtag(tag):
    return pattern.findall(tag)

In [ ]:
list(map(split_hashtag, hashtags))

In [ ]:
!pip install tweepy

In [ ]:
import tweepy

auth = tweepy.OAuthHandler('XDZYetUApxdI3hnrYWcFZbX2w', 'r7nWPIRwhXaxsoCNVjyMtFGPJBEIlpoZoePKRH60mL3MJ2rtsf')
auth.set_access_token('314861452-PFLPbE8Mgkt4XF1VRpA0tJGvh4sXyM08uPm3oTX9', 'k5NbfAh83UuKothMnc97mN8MIlGdY7AlzI0lvzhhEPNs3')

api = tweepy.API(auth)

user = api.get_user('twitter')

In [ ]:
user._json.keys()

In [ ]:
from tweepy import TweepError
#users_info = {}
rejected = []
for user in users:
    if user in rejected:
        try:    
            users_info[user] = api.get_user(user)._json
        except TweepError as e:
            if (e.api_code!= 50) and (e.api_code != 63):
                users_info[user] = {}
            else:
                rejected.append(user)
            print(e)

In [ ]:
user = api.get_user('nocdknjcfdknjdfhbkvfdhfdv')

In [ ]:
users_in = [user for user in users_info.keys() if users_info[user].get('id', False)]

In [ ]:
len(users_in)

In [ ]:
with open(sst_home + '/saved_data/twitter_data.pickle', 'wb') as handle:
    pickle.dump(users_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Error Analysis

In [ ]:
validation_data = {}
with open(sst_home + '/profner-data/subtask-2/tsv/valid.tsv', 'r') as file:
    for line in file.readlines()[1:]:
        t = line.split('\t')
        validation_data[t[0]] = validation_data.get(t[0], []) + [(t[1], t[2], t[3], t[4][:-1])]

prediction_data = {}
with open(sst_home + '/final-result/results.tsv', 'r') as file:
    for line in file.readlines()[1:]:
        t = line.split('\t')
        prediction_data[t[0]] = prediction_data.get(t[0], []) + [(t[1], t[2], t[3], t[4][:-1])]

In [ ]:
for tweet in validation_data.keys():
    print(f'######## Tweet: {tweet} ########')
    print('######## Validation ########')
    print(validation_data[tweet])
    print('######## Prediction ########')
    print(prediction_data.get(tweet, {}))
    print()
    print()
    
    
    input('')

In [ ]:
!pip install rae

In [ ]:
import rae

In [ ]:
rae.DLE.search_word('capellán')

In [ ]:
!pip install pyrae

In [ ]:
translate_text = translator.translate('polcia',lang_src='es',lang_tgt='en')
translate_text = translator.translate(translate_text,lang_src='en',lang_tgt='es')
print(translate_text)

In [ ]:
getDictEntities(sst_home + '/profner-data/subtask-2/brat/valid/1254776314545836032.ann' )

In [ ]:
tag2idx

## Data Augmentation

The purpose of this section is to generate new training data based on existing data by using the similarity vectors to replace entities with synonyms.

In [ ]:
### SPANISH MEDICAL CORPORA ###
sst_home_embeddings = sst_home + '/fast-text-model/'
ft = fasttext.load_model(sst_home_embeddings + 'covid_19_es_twitter_cbow_cased.bin')

In [ ]:
for result in ft.get_nearest_neighbors('gobierno', k=10):
    print(nlp(result[1])[0].ipy result[1])

In [18]:
SINGLE = 'S'
BEGIN = 'B'
END = 'E'
INSIDE = 'I'
OUT = 'O'

def annotateV2(y_pred, offset, file_name, extra = None):    
    max_index = len(y_pred)
    if '#ENDPAD' in y_pred:
        max_index = y_pred.index('#ENDPAD')
        
    right_moves = {'B': [ 'I', 'E'],
                   'I': ['I', 'E'],
                   'E': ['B', 'S', 'O'],
                   'S': ['B', 'S', 'O'],
                   'O': ['B', 'S', 'O']}
    text = ''
    entity = ''
    start = -1
    end = -1
    
    has_error = False
    for i in range(max_index - 1):
        if not (has_error and y_pred[i][0] in ['I','E']):
            has_error = False
            ann = y_pred[i][0]
            info = offset[i]
            next_ann = y_pred[i + 1][0]
            entity_class = y_pred[i][2:]
            ### SI LA EL MOVIMIENTO ES PERMITIDO, SE EJECUTA SU POLITICA NORMAL
            if next_ann in right_moves[ann]:
                info = offset[i]
                if ann == BEGIN:
                    entity = info[0]
                    start = info[1]
                    end = info[2]
                if ann == INSIDE:
                    entity = entity + ' ' + info[0]
                    end = info[2]
                if ann == END:
                    entity = entity + ' ' + info[0]
                    end = info[2]
                    text = text + f'{file_name}\t{start}\t{end}\t{entity_class}\t{entity}\n'
                    entity = ''
                    start = -1
                    end = -1
                if ann == SINGLE:
                    text = text + f'{file_name}\t{info[1]}\t{info[2]}\t{entity_class}\t{info[0]}\n'
                    entity = ''
                    start = -1
                    end = -1
                if ann == OUT:
                    entity = ''
                    start = -1
                    end = -1
            else:
                has_error = True
                
                file_ann = sst_home + f'/profner-data/subtask-2/brat/valid/{file_name}.ann' 
                with open(file_ann) as file:
                  print(file.read())
                
                '''
                if ann == 'I' and next_ann == 'O':
                    entity = " ".join([entity, info[0], offset[i + 1][0]])
                    print(f'{file_name}\t{start}\t{offset[i + 1][2]}\t{entity_class}\t{entity}\n')
                    text = text + f'{file_name}\t{start}\t{offset[i + 1][2]}\t{entity_class}\t{entity}\n'
                '''
                
                if ann == 'B' and next_ann == 'O':
                    print('###### ERROR #######')
                    print(f'Error: {ann} -> {next_ann} -> {y_pred[i + 2]}')
                    if ann != OUT:
                        info = offset[i]
                        if entity == '':
                            print(f'Entidad: "{info[0]}" Clase: "{y_pred[i][2:]}"')
                        else:
                            print(f'Entidad: {entity  + info[0] + offset[i + 1][0]} Clase: {y_pred[i][2:]} Offset: ({start},{info[2]})')
                    if next_ann != OUT:
                        pass
                    print(f'Siguiente Entidad {offset[i + 1][0]}')
                    print()
                    
                entity = ''
                start = -1
                end = -1
            
    return text  

In [ ]:
nlp('#Camión')[1]